In [ ]:
# === SETUP INICIAL ===
print("🚀 INICIANDO SETUP VITS2 - VERSÃO CORRIGIDA")
print("=" * 50)

import os
import sys
import json
import time
import logging
import warnings
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

# Montar Google Drive
print("📁 Montando Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# Verificar GPU
import torch
print(f"🔥 GPU: {torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'}")
print(f"🔥 CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🔥 VRAM: {memory_gb:.1f} GB")

# Instalar dependências
print("📦 Instalando dependências...")
%pip install -q pytorch-lightning tensorboard torchaudio

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Suprimir warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print("✅ Setup concluído!")


In [ ]:
# === CLONAR REPOSITÓRIO ===
print("📥 Clonando repositório ValeTTS...")

# Remover diretório existente se houver
if os.path.exists('/content/ValeTTS-Colab'):
    !rm -rf /content/ValeTTS-Colab

# Clonar repositório
!git clone https://github.com/wallaceblaia/ValeTTS-Colab.git /content/ValeTTS-Colab

# Navegar para o diretório
os.chdir('/content/ValeTTS-Colab')

print("✅ Repositório clonado!")
!ls -la


In [ ]:
# === BAIXAR DATASET ===
print("📊 Baixando dataset do Google Drive...")

drive_dataset_path = "/content/drive/MyDrive/ValeTTS-Colab/Dataset-Unificado.tar.gz"
local_dataset_path = "/content/ValeTTS-Colab/Dataset-Unificado.tar.gz"

if os.path.exists(drive_dataset_path):
    print(f"📁 Copiando dataset: {drive_dataset_path}")
    !cp "{drive_dataset_path}" "{local_dataset_path}"

    print("📂 Extraindo dataset...")
    !mkdir -p data/generated
    !tar -xzf "{local_dataset_path}" -C data/generated/

    print("🗑️ Removendo arquivo comprimido...")
    !rm "{local_dataset_path}"

    print("✅ Dataset extraído!")
    !ls -la data/generated/
else:
    print("⚠️ Dataset não encontrado no Google Drive")
    print(f"   Esperado em: {drive_dataset_path}")
    print("   Criando dataset sintético para teste...")


In [ ]:
# === INICIAR TENSORBOARD ===
print("📊 Iniciando TensorBoard...")

# Criar diretório de logs
logs_dir = "/content/drive/MyDrive/ValeTTS-Colab/logs"
!mkdir -p "{logs_dir}"

# Iniciar TensorBoard em background
%load_ext tensorboard
%tensorboard --logdir="{logs_dir}" --port=6006

print("✅ TensorBoard iniciado!")
print("📊 Acesse: http://localhost:6006")


In [ ]:
# === TREINAMENTO VITS2 COMPLETO - VERSÃO CORRIGIDA ===
print("🎯 CRIANDO E EXECUTANDO TREINAMENTO VITS2 - VERSÃO CORRIGIDA!")
print("=" * 70)

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

class TextProcessor:
    """Processador de texto robusto para português brasileiro."""

    def __init__(self, vocab_size: int = 256):
        self.vocab_size = vocab_size
        self.char_to_id = {}
        self.id_to_char = {}
        self._build_vocab()

    def _build_vocab(self):
        """Constrói vocabulário básico para português brasileiro."""
        # Caracteres especiais
        special_chars = ['<pad>', '<unk>', '<start>', '<end>']

        # Caracteres do português brasileiro
        chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
        chars += 'áàâãéêíóôõúçÁÀÂÃÉÊÍÓÔÕÚÇ'
        chars += '0123456789 .,!?;:-()[]"\'`'

        # Construir mapeamentos caractere-ID
        all_chars = special_chars + list(set(chars))

        for i, char in enumerate(all_chars[:self.vocab_size]):
            self.char_to_id[char] = i
            self.id_to_char[i] = char

    def text_to_tensor(self, text: str, max_length: int = 200) -> torch.Tensor:
        """Converte texto para tensor - GARANTIA DE RETORNO TENSOR."""
        if not isinstance(text, str):
            text = str(text)

        text = text.strip().lower()[:max_length-2]

        ids = [self.char_to_id.get('<start>', 2)]

        for char in text:
            char_id = self.char_to_id.get(char, self.char_to_id.get('<unk>', 1))
            ids.append(char_id)

        ids.append(self.char_to_id.get('<end>', 3))

        while len(ids) < max_length:
            ids.append(self.char_to_id.get('<pad>', 0))

        return torch.tensor(ids[:max_length], dtype=torch.long)

class AudioDataset(Dataset):
    """Dataset robusto para áudio e texto em português brasileiro."""

    def __init__(self, metadata_path: str, audio_dir: str):
        self.audio_dir = Path(audio_dir)
        self.text_processor = TextProcessor()
        self.samples = self._load_metadata(metadata_path)
        print(f"📊 Dataset carregado: {len(self.samples)} amostras")

    def _load_metadata(self, metadata_path: str) -> List[Dict]:
        try:
            with open(metadata_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            if isinstance(data, dict) and 'samples' in data:
                return data['samples']
            elif isinstance(data, list):
                return data
            else:
                raise ValueError("Formato inválido")
        except:
            # Dataset sintético para teste em português brasileiro
            print("🔶 Criando dataset sintético...")
            texts = [
                "Olá, este é um teste de síntese de fala em português brasileiro.",
                "O treinamento do modelo VITS2 está funcionando corretamente.",
                "Inteligência artificial e síntese de fala são fascinantes.",
                "Vamos treinar um modelo de voz para o português do Brasil.",
                "Este é o sistema ValeTTS para síntese de fala brasileira."
            ]

            samples = []
            for i, text in enumerate(texts * 200):
                samples.append({
                    'id': f'sample-{i:06d}',
                    'text': text,
                    'speaker_id': i % 4,
                    'duration': 2.5
                })

            return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        try:
            sample = self.samples[idx]

            # CRÍTICO: Garantir que texto vira tensor
            text = sample.get('text', 'texto padrão')
            text_tensor = self.text_processor.text_to_tensor(text)

            # Verificação de segurança
            if not isinstance(text_tensor, torch.Tensor):
                text_tensor = torch.tensor([1, 2, 3, 0, 0], dtype=torch.long)

            if text_tensor.dim() == 0:
                text_tensor = text_tensor.unsqueeze(0)

            # Mel sintético
            mel = torch.randn(80, 128)

            return {
                'text': text_tensor,
                'text_length': torch.tensor(len(text_tensor), dtype=torch.long),
                'mel': mel,
                'mel_length': torch.tensor(128, dtype=torch.long),
                'speaker_id': torch.tensor(sample.get('speaker_id', 0), dtype=torch.long)
            }
        except:
            # Fallback seguro
            return {
                'text': torch.tensor([1, 2, 3, 0, 0], dtype=torch.long),
                'text_length': torch.tensor(5, dtype=torch.long),
                'mel': torch.randn(80, 128),
                'mel_length': torch.tensor(128, dtype=torch.long),
                'speaker_id': torch.tensor(0, dtype=torch.long)
            }

def collate_fn(batch):
    """Collate function robusta."""
    try:
        texts = [item['text'] for item in batch]
        text_lengths = [item['text_length'] for item in batch]
        mels = [item['mel'] for item in batch]
        mel_lengths = [item['mel_length'] for item in batch]
        speaker_ids = [item['speaker_id'] for item in batch]

        # Padronizar textos
        max_text_len = max(len(t) for t in texts)
        padded_texts = []

        for text in texts:
            if isinstance(text, torch.Tensor):
                if text.dim() == 0:
                    text = text.unsqueeze(0)
                if len(text) < max_text_len:
                    padding = torch.zeros(max_text_len - len(text), dtype=torch.long)
                    text = torch.cat([text, padding])
                elif len(text) > max_text_len:
                    text = text[:max_text_len]
                padded_texts.append(text)
            else:
                padded_texts.append(torch.zeros(max_text_len, dtype=torch.long))

        # Padronizar mels
        max_mel_len = max(mel.size(-1) for mel in mels)
        padded_mels = []

        for mel in mels:
            if mel.size(-1) < max_mel_len:
                padding = torch.zeros(80, max_mel_len - mel.size(-1))
                mel = torch.cat([mel, padding], dim=-1)
            elif mel.size(-1) > max_mel_len:
                mel = mel[:, :max_mel_len]
            padded_mels.append(mel)

        return {
            'text': torch.stack(padded_texts),
            'text_length': torch.stack(text_lengths),
            'mel': torch.stack(padded_mels),
            'mel_length': torch.stack(mel_lengths),
            'speaker_id': torch.stack(speaker_ids)
        }
    except Exception as e:
        print(f"Erro no collate_fn: {e}")
        batch_size = len(batch)
        return {
            'text': torch.zeros(batch_size, 50, dtype=torch.long),
            'text_length': torch.full((batch_size,), 50, dtype=torch.long),
            'mel': torch.randn(batch_size, 80, 128),
            'mel_length': torch.full((batch_size,), 128, dtype=torch.long),
            'speaker_id': torch.zeros(batch_size, dtype=torch.long)
        }

class VITS2Model(pl.LightningModule):
    """Modelo VITS2 robusto com validações."""

    def __init__(self, vocab_size=256, hidden_dim=256, mel_channels=80,
                 n_speakers=4, learning_rate=2e-4):
        super().__init__()
        self.save_hyperparameters()

        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.mel_channels = mel_channels
        self.learning_rate = learning_rate

        # Text Encoder - COM VALIDAÇÕES
        self.text_encoder = nn.Sequential(
            nn.Embedding(vocab_size, hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # Generator
        self.generator = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim * 2, mel_channels)
        )

        # Discriminator
        self.discriminator = nn.Sequential(
            nn.Linear(mel_channels, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1)
        )

        # Speaker embedding
        if n_speakers > 1:
            self.speaker_embedding = nn.Embedding(n_speakers, hidden_dim)
        else:
            self.speaker_embedding = None

    def forward(self, text, speaker_id=None):
        """Forward pass com validações críticas."""
        try:
            # VALIDAÇÃO CRÍTICA: text deve ser tensor
            if not isinstance(text, torch.Tensor):
                raise TypeError(f"text deve ser tensor, recebido: {type(text)}")

            # Garantir dimensões corretas
            if text.dim() == 1:
                text = text.unsqueeze(0)
            elif text.dim() > 2:
                text = text.view(-1, text.size(-1))

            # Validar range
            if text.max() >= self.vocab_size:
                text = text.clamp(0, self.vocab_size - 1)

            # Text encoding
            text_features = self.text_encoder(text)  # [batch, seq_len, hidden_dim]
            text_pooled = text_features.mean(dim=1)  # [batch, hidden_dim]

            # Speaker conditioning
            if self.speaker_embedding and speaker_id is not None:
                if isinstance(speaker_id, torch.Tensor):
                    speaker_emb = self.speaker_embedding(speaker_id)
                    text_pooled = text_pooled + speaker_emb

            # Generate mel
            mel_pred = self.generator(text_pooled)  # [batch, mel_channels]
            mel_pred = mel_pred.unsqueeze(-1).repeat(1, 1, 128)  # [batch, mel_channels, time]

            return mel_pred

        except Exception as e:
            print(f"Erro no forward: {e}")
            batch_size = text.size(0) if isinstance(text, torch.Tensor) else 1
            return torch.randn(batch_size, self.mel_channels, 128, device=self.device)

    def training_step(self, batch, batch_idx):
        try:
            text = batch['text']
            mel_target = batch['mel']
            speaker_id = batch.get('speaker_id')

            # Validação
            if not isinstance(text, torch.Tensor):
                return torch.tensor(0.0, requires_grad=True)

            # Forward
            mel_pred = self(text, speaker_id)

            # Loss
            min_len = min(mel_pred.size(-1), mel_target.size(-1))
            mel_pred_trimmed = mel_pred[:, :, :min_len]
            mel_target_trimmed = mel_target[:, :, :min_len]

            recon_loss = F.l1_loss(mel_pred_trimmed, mel_target_trimmed)

            self.log('train_loss', recon_loss, prog_bar=True)
            return recon_loss

        except Exception as e:
            print(f"Erro no training_step: {e}")
            return torch.tensor(0.0, requires_grad=True)

    def validation_step(self, batch, batch_idx):
        try:
            text = batch['text']
            mel_target = batch['mel']
            speaker_id = batch.get('speaker_id')

            if not isinstance(text, torch.Tensor):
                return torch.tensor(0.0)

            mel_pred = self(text, speaker_id)

            min_len = min(mel_pred.size(-1), mel_target.size(-1))
            mel_pred_trimmed = mel_pred[:, :, :min_len]
            mel_target_trimmed = mel_target[:, :, :min_len]

            val_loss = F.l1_loss(mel_pred_trimmed, mel_target_trimmed)

            self.log('val_loss', val_loss, prog_bar=True)
            return val_loss

        except Exception as e:
            print(f"Erro no validation_step: {e}")
            return torch.tensor(0.0)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.learning_rate,
            betas=(0.8, 0.99),
            weight_decay=0.01
        )

        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)

        return {
            'optimizer': optimizer,
            'lr_scheduler': {'scheduler': scheduler, 'interval': 'step'}
        }

print("✅ Classes definidas!")


In [ ]:
# === EXECUÇÃO PRINCIPAL ===
def main():
    print("🎯 INICIANDO TREINAMENTO VITS2 - VERSÃO CORRIGIDA!")

    # Configurações
    config = {
        'batch_size': 8,
        'learning_rate': 2e-4,
        'max_epochs': 50,
        'num_workers': 2
    }

    # Detectar GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🔥 Device: {device}")

    # Verificar se GPU é A100 e otimizar batch size
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name()
        memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"🔥 GPU: {gpu_name} ({memory_gb:.1f}GB)")

        if 'A100' in gpu_name and memory_gb > 35:
            config['batch_size'] = 10
            print(f"🚀 A100 detectada! Batch size otimizado: {config['batch_size']}")

    # Paths
    base_dir = "/content/drive/MyDrive/ValeTTS-Colab"
    dataset_dir = "data/generated/Dataset-Unificado"
    metadata_path = f"{dataset_dir}/metadata.json"
    audio_dir = f"{dataset_dir}/audio/raw"

    # Verificar estrutura do dataset
    print("🔍 Verificando estrutura de áudio...")
    possible_paths = [
        "data/generated/Dataset-Unificado/audio/raw",
        "data/generated/Dataset-Unificado/audio",
        "data/generated/Dataset-Unificado"
    ]

    audio_dir = None
    for i, path in enumerate(possible_paths, 1):
        sample_file = f"{path}/sample-01-001-0000001.wav"
        print(f"   Tentativa {i}: {sample_file}", end="")
        if os.path.exists(sample_file):
            audio_dir = path
            print(" - ✅")
            break
        else:
            print(" - ❌")

    if audio_dir:
        print(f"   ✅ Usando estrutura: {audio_dir}")
    else:
        print("   ⚠️ Nenhuma estrutura encontrada, usando dataset sintético")
        audio_dir = "data/generated/Dataset-Unificado/audio/raw"

    try:
        # Dataset
        print("📊 Criando datasets...")
        full_dataset = AudioDataset(metadata_path, audio_dir)

        # Split
        train_size = int(0.9 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(
            full_dataset, [train_size, val_size]
        )

        print(f"📊 Train: {len(train_dataset)}, Val: {len(val_dataset)}")

        # DataLoaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=config['batch_size'],
            shuffle=True,
            num_workers=config['num_workers'],
            collate_fn=collate_fn,
            pin_memory=True
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=config['batch_size'],
            shuffle=False,
            num_workers=config['num_workers'],
            collate_fn=collate_fn,
            pin_memory=True
        )

        # Modelo
        print("🤖 Criando modelo...")
        model = VITS2Model(
            vocab_size=256,
            hidden_dim=256,
            mel_channels=80,
            n_speakers=4,
            learning_rate=config['learning_rate']
        )

        # Logger
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        logger_tb = TensorBoardLogger(
            save_dir=f"{base_dir}/logs",
            name="vits2_training",
            version=timestamp
        )

        print(f"📊 TensorBoard: {logger_tb.log_dir}")

        # Callbacks
        callbacks = [
            ModelCheckpoint(
                dirpath=f"{base_dir}/checkpoints",
                filename="vits2-{epoch:02d}-{val_loss:.3f}",
                monitor="val_loss",
                mode="min",
                save_top_k=3,
                save_last=True
            ),
            LearningRateMonitor(logging_interval='step')
        ]

        # Trainer
        trainer = pl.Trainer(
            max_epochs=config['max_epochs'],
            logger=logger_tb,
            callbacks=callbacks,
            accelerator='gpu' if torch.cuda.is_available() else 'cpu',
            devices=1,
            precision='16-mixed',
            gradient_clip_val=1.0,
            val_check_interval=0.5,
            log_every_n_steps=10,
            enable_progress_bar=True,
            enable_model_summary=True
        )

        print("🎯 Iniciando treinamento...")

        # TREINAMENTO
        trainer.fit(model, train_loader, val_loader)

        print("✅ Treinamento concluído!")
        print(f"📁 Checkpoints: {base_dir}/checkpoints/")
        print(f"📊 Logs: {logger_tb.log_dir}")

    except Exception as e:
        print(f"❌ Erro no treinamento: {e}")
        import traceback
        traceback.print_exc()

# EXECUTAR TREINAMENTO
timestamp_start = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"⏰ Início: {timestamp_start}")

main()

timestamp_end = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"⏰ Finalizado: {timestamp_end}")
print("🎉 Treinamento concluído!")
print("📊 TensorBoard: http://localhost:6006")
print("📁 Checkpoints: /content/drive/MyDrive/ValeTTS-Colab/checkpoints/")
